In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [8]:
response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            tools=tools
        )

In [17]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DK57AqiCn47IgTrc0NmJWYG9', function=Function(arguments='{\n  "location": "Boston, MA",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])

The message does not have content, only the function to use and parameters. Developers are responsible for running the function on their own. 

In [22]:
function_response = response_message.tool_calls[0].function

In [26]:
function_arguments = function_response.arguments
args = json.loads(function_arguments)

In [27]:
get_current_weather(args)

'{"location": {"location": "Boston, MA", "format": "celsius"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

Let`s see what happens, when the prompt does not have anything in common with the function. 

In [28]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [29]:
response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            tools=tools
        )

In [30]:
print(response)

ChatCompletion(id='chatcmpl-96zBXYnWqKYyb6Q8zJQo3Ph2hJMQ9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1711452191, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=86, total_tokens=96))


Model determined, that it is not needed to use the function. 

In [40]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            tools=tools,  
            tool_choice={"type": "function", "function": {"name": "get_current_weather"}} 
        )

ChatCompletion(id='chatcmpl-970Wa4vNi0MyoIrUZAzHbxGIW9oYn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yE5CIKKRRmqqZ7nhLKKEsyBw', function=Function(arguments='{\n  "location": "San Francisco, CA",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')]))], created=1711457340, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=93, total_tokens=113))


In [43]:
response.choices[0].message.tool_calls[0].function.arguments

'{\n  "location": "San Francisco, CA",\n  "format": "celsius"\n}'

In [36]:
response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            tools=tools,  
            tool_choice="none" 
        )

In [37]:
response.choices[0].message.content

'Hello! How can I assist you today?'